## Loan Deliqency

In [13]:
import numpy as np
import pandas as pd
import warnings; warnings.simplefilter('ignore')
from sklearn.ensemble import RandomForestClassifier

In [14]:
train = pd.read_csv("train.csv")

In [15]:
train.head()

,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,...,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13
0,268055008619,Z,"Turner, Baldwin and Rhodes",4.250,214000,360,2012-03-01,05/2012,95,1.0,...,0,0,0,1,0,0,0,0,0,1
1,672831657627,Y,"Swanson, Newton and Miller",4.875,144000,360,2012-01-01,03/2012,72,1.0,...,0,0,0,0,0,0,0,1,0,1
2,742515242108,Z,Thornton-Davis,3.250,366000,180,2012-01-01,03/2012,49,1.0,...,0,0,0,0,0,0,0,0,0,1
3,601385667462,X,OTHER,4.750,135000,360,2012-02-01,04/2012,46,2.0,...,0,0,0,0,0,1,1,1,1,1
4,273870029961,X,OTHER,4.750,124000,360,2012-02-01,04/2012,80,1.0,...,3,4,5,6,7,8,9,10,11,1


In [16]:
y = pd.DataFrame(train['m13'])
train.isnull().sum()

loan_id                     0
source                      0
financial_institution       0
interest_rate               0
unpaid_principal_bal        0
loan_term                   0
origination_date            0
first_payment_date          0
loan_to_value               0
number_of_borrowers         0
debt_to_income_ratio        0
borrower_credit_score       0
loan_purpose                0
insurance_percent           0
co-borrower_credit_score    0
insurance_type              0
m1                          0
m2                          0
m3                          0
m4                          0
m5                          0
m6                          0
m7                          0
m8                          0
m9                          0
m10                         0
m11                         0
m12                         0
m13                         0
dtype: int64

In [17]:
print(train.shape)

(116058, 29)


In [18]:
train["m13"].value_counts()

0    115422
1       636
Name: m13, dtype: int64

In [19]:
train.dtypes

loan_id                       int64
source                       object
financial_institution        object
interest_rate               float64
unpaid_principal_bal          int64
loan_term                     int64
origination_date             object
first_payment_date           object
loan_to_value                 int64
number_of_borrowers         float64
debt_to_income_ratio        float64
borrower_credit_score       float64
loan_purpose                 object
insurance_percent           float64
co-borrower_credit_score    float64
insurance_type              float64
m1                            int64
m2                            int64
m3                            int64
m4                            int64
m5                            int64
m6                            int64
m7                            int64
m8                            int64
m9                            int64
m10                           int64
m11                           int64
m12                         

In [20]:
train['credit_score'] = train['borrower_credit_score'] + train['co-borrower_credit_score']

In [21]:
X1 = train.drop(['loan_id', 'first_payment_date','origination_date','m13','insurance_percent','insurance_type'],axis=1)


Here by label encoder I convert object values to numerical values 

In [22]:
from sklearn.preprocessing import LabelEncoder
lm = LabelEncoder()
a  = ['financial_institution']
for i in np.arange(len(a)):
    X1[a[i]] = lm.fit_transform(X1[a[i]])
    


By one hot encoding I convert 'source','loan_purpose' into dummy variables

In [23]:
sourc = pd.get_dummies(X1['source'])
LoanP = pd.get_dummies(X1['loan_purpose'])
X1 = pd.concat([X1,LoanP],axis = 1)
X1 = X1.drop(['source','loan_purpose'],axis=1)


In [24]:
print(X1.shape)


(116058, 25)


In [25]:
X1.head()

,financial_institution,interest_rate,unpaid_principal_bal,loan_term,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,co-borrower_credit_score,m1,...,m7,m8,m9,m10,m11,m12,credit_score,A23,B12,C86
0,18,4.250,214000,360,95,1.0,22.0,694.0,0.0,0,...,1,0,0,0,0,0,694.0,0,0,1
1,15,4.875,144000,360,72,1.0,44.0,697.0,0.0,0,...,0,0,0,0,1,0,697.0,0,1,0
2,17,3.250,366000,180,49,1.0,33.0,780.0,0.0,0,...,0,0,0,0,0,0,780.0,0,1,0
3,8,4.750,135000,360,46,2.0,44.0,633.0,638.0,0,...,0,0,1,1,1,1,1271.0,0,1,0
4,8,4.750,124000,360,80,1.0,43.0,681.0,0.0,0,...,6,7,8,9,10,11,681.0,0,0,1


In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.3, random_state=2)

In [28]:
rft=RandomForestClassifier(random_state=42,max_depth= 10 ,min_samples_split= 5 ,n_estimators= 500,n_jobs=-1, min_samples_leaf=2,  criterion='entropy',class_weight= {0: 0.8, 1:1})
rft.fit(X_train,y_train) 

RandomForestClassifier(bootstrap=True, class_weight={0: 0.8, 1: 1},
                       criterion='entropy', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=2,
                       min_samples_split=5, min_weight_fraction_leaf=0.0,
                       n_estimators=500, n_jobs=-1, oob_score=False,
                       random_state=42, verbose=0, warm_start=False)

In [29]:
from sklearn.metrics import f1_score,confusion_matrix

In [30]:
y_pred=rft.predict(X_test)

print(f1_score(y_test,y_pred))

confusion_matrix(y_test,y_pred)

0.4827586206896552


array([[34598,    21],
       [  129,    70]])

In [31]:
threshold = 0.26
y_pred_prob = rft.predict_proba(X_test)[:,1]
y_pred = (y_pred_prob > threshold).astype(int)

print(f1_score(y_test,y_pred))

confusion_matrix(y_test,y_pred)

In [1]:
rft.feature_importances_

NameError: name 'rft' is not defined

In [33]:
pd.Series(rft.feature_importances_,index=X1.columns).sort_values(ascending=False)

m12                         0.273890
m11                         0.149787
m9                          0.077019
m10                         0.071644
borrower_credit_score       0.067513
credit_score                0.051844
m8                          0.047551
m7                          0.035586
interest_rate               0.029349
debt_to_income_ratio        0.027591
unpaid_principal_bal        0.026453
co-borrower_credit_score    0.025587
m5                          0.023365
loan_to_value               0.020697
m4                          0.014474
m6                          0.014400
financial_institution       0.012353
loan_term                   0.006098
m1                          0.005505
A23                         0.003965
C86                         0.003562
B12                         0.003335
number_of_borrowers         0.003189
m2                          0.002888
m3                          0.002355
dtype: float64

I hyper tuned my random forest classifier by varying depth between [6,15] and min sample split between[2,5]. I used criterion as "Entropy" because entropy has an edge in some data cases involving a high imbalance. 
I have given class weight { 0:0.8, 1:1 } to minimize 'class1' error which means "It is ok that If I don't give loan to 2-3 good guys but I don't want to give loan to those people who can be loan defaulters".

In [34]:
# XGB Classifier
from xgboost import XGBClassifier

classifier = XGBClassifier( learning_rate =0.1,
 n_estimators=112,
 max_depth=9,
 min_child_weight=5,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.6,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=13,
 reg_lambda=5,
# max_delta_step=1,
 alpha=0,
 base_score=0.5,
 seed=1029)

classifier.fit(X_train, y_train)


XGBClassifier(alpha=0, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=9,
              min_child_weight=5, missing=None, n_estimators=112, n_jobs=1,
              nthread=4, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=5, scale_pos_weight=13, seed=1029,
              silent=None, subsample=0.8, verbosity=1)

In [35]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred



f1_score(y_test, y_pred)

array([0, 0, 0, ..., 0, 0, 0])

In [67]:
threshold = 0.47
y_pred_prob = classifier.predict_proba(X_test)[:,1]
y_pred = (y_pred_prob > threshold).astype(int)

print(f1_score(y_test,y_pred))

confusion_matrix(y_test,y_pred)

0.5230769230769231


array([[34530,    89],
       [   97,   102]])

In [96]:
from xgboost import XGBClassifier
classifier = XGBClassifier( learning_rate =0.1,
 n_estimators=100,
 max_depth=32,
 min_child_weight=5,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.6,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=13,
 reg_lambda=5,
# max_delta_step=1,
 alpha=0,
 base_score=0.5,
 seed=1029)

classifier.fit(X_train, y_train)

XGBClassifier(alpha=0, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=32,
              min_child_weight=5, missing=None, n_estimators=100, n_jobs=1,
              nthread=4, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=5, scale_pos_weight=13, seed=1029,
              silent=None, subsample=0.8, verbosity=1)

In [97]:
threshold = 0.47
y_pred_prob = classifier.predict_proba(X_test)[:,1]
y_pred = (y_pred_prob > threshold).astype(int)

print(f1_score(y_test,y_pred))

confusion_matrix(y_test,y_pred)

0.5424657534246575


array([[34552,    67],
       [  100,    99]])

In [98]:
# XGB Classifier
from xgboost import XGBClassifier

classifier = XGBClassifier( learning_rate =0.1,
 n_estimators=90,
 max_depth=32,
 min_child_weight=5,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.6,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=13,
 reg_lambda=5,
# max_delta_step=1,
 alpha=0,
 base_score=0.5,
 seed=1029)

classifier.fit(X_train, y_train)

XGBClassifier(alpha=0, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=32,
              min_child_weight=5, missing=None, n_estimators=90, n_jobs=1,
              nthread=4, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=5, scale_pos_weight=13, seed=1029,
              silent=None, subsample=0.8, verbosity=1)

In [112]:
threshold = 0.51
y_pred_prob = classifier.predict_proba(X_test)[:,1]
y_pred = (y_pred_prob > threshold).astype(int)

print(f1_score(y_test,y_pred))

confusion_matrix(y_test,y_pred)

0.5524861878453038


array([[34556,    63],
       [   99,   100]])

In [126]:
# XGB Classifier
from xgboost import XGBClassifier

classifier = XGBClassifier( learning_rate =0.1,
 n_estimators=91,
 max_depth=32,
 min_child_weight=5,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.6,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=13,
 reg_lambda=5,
# max_delta_step=1,
 alpha=0,
 base_score=0.5,
 seed=1029)

classifier.fit(X_train, y_train)

XGBClassifier(alpha=0, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=32,
              min_child_weight=5, missing=None, n_estimators=91, n_jobs=1,
              nthread=4, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=5, scale_pos_weight=13, seed=1029,
              silent=None, subsample=0.8, verbosity=1)

In [127]:
threshold = 0.51
y_pred_prob = classifier.predict_proba(X_test)[:,1]
y_pred = (y_pred_prob > threshold).astype(int)

print(f1_score(y_test,y_pred))

confusion_matrix(y_test,y_pred)

0.5555555555555556


array([[34558,    61],
       [   99,   100]])

In [178]:
from sklearn.ensemble import GradientBoostingClassifier

In [179]:
gbc = GradientBoostingClassifier (loss='deviance', learning_rate=0.1, n_estimators=1000, min_samples_split=2, min_samples_leaf=1, max_depth=3, random_state=2,verbose=2 ) 

In [180]:
gbc.fit(X_train,y_train)

      Iter       Train Loss   Remaining Time 
         1           0.0470            5.05m
         2           0.0458            3.61m
         3           0.0450            2.80m
         4           0.0445            2.62m
         5           0.0438            2.29m
         6           0.0432            2.24m
         7           0.0427            2.22m
         8           0.0424            2.16m
         9           0.1089            2.10m
        10         724.0756            2.02m
        11       55248.2728            1.98m
        12       55248.2709            1.91m
        13       55248.2694            1.87m
        14       55248.2678            1.83m
        15       55248.2667            1.80m
        16       55248.2611            1.76m
        17       55230.8491            1.73m
        18       55230.8490            1.70m
        19       55230.8490            1.66m
        20       55230.8490            1.63m
        21       55230.8489            1.62m
        2

       183       55230.8466            1.26m
       184       55230.8466            1.26m
       185       55230.8466            1.25m
       186       55230.8466            1.25m
       187       55230.8466            1.25m
       188       55230.8466            1.25m
       189       55230.8466            1.24m
       190       55230.8466            1.24m
       191       55230.8466            1.23m
       192       55230.8466            1.23m
       193       55230.8466            1.23m
       194       55230.8466            1.22m
       195       55230.8466            1.22m
       196       55230.8466            1.22m
       197       55230.8466            1.21m
       198       55230.8466            1.21m
       199       55230.8466            1.21m
       200       55230.8466            1.20m
       201       55230.8466            1.20m
       202       55230.8466            1.20m
       203       55230.8466            1.20m
       204       55230.8466            1.19m
       205

       366       55230.8466           55.95s
       367       55230.8466           55.90s
       368       55230.8466           55.77s
       369       55230.8466           55.66s
       370       55230.8466           55.53s
       371       55230.8466           55.38s
       372       55230.8466           55.25s
       373       55230.8466           55.15s
       374       55230.8466           55.01s
       375       55230.8466           54.90s
       376       55230.8466           54.77s
       377       55230.8466           54.68s
       378       55230.8466           54.54s
       379       55230.8466           54.40s
       380       55230.8466           54.26s
       381       55230.8466           54.16s
       382       55230.8466           54.03s
       383       55230.8466           53.93s
       384       55230.8466           53.80s
       385       55230.8466           53.71s
       386       55230.8466           53.57s
       387       55230.8466           53.44s
       388

       549       55230.8466           39.50s
       550       55230.8466           39.40s
       551       55230.8466           39.30s
       552       55230.8466           39.22s
       553       55230.8466           39.11s
       554       55230.8466           39.00s
       555       55230.8466           38.89s
       556       55230.8466           38.79s
       557       55230.8466           38.69s
       558       55230.8466           38.58s
       559       55230.8466           38.48s
       560       55230.8466           38.41s
       561       55230.8466           38.31s
       562       55230.8466           38.21s
       563       55230.8466           38.11s
       564       55230.8466           38.06s
       565       55230.8466           37.95s
       566       55230.8466           37.88s
       567       55230.8466           37.77s
       568       55230.8466           37.67s
       569       55230.8466           37.57s
       570       55230.8466           37.47s
       571

       735       55230.8466           22.24s
       736       55230.8466           22.14s
       737       55230.8466           22.05s
       738       55230.8466           21.96s
       739       55230.8466           21.88s
       740       55230.8466           21.78s
       741       55230.8466           21.69s
       742       55230.8466           21.60s
       743       55230.8466           21.52s
       744       55230.8466           21.43s
       745       55230.8466           21.34s
       746       55230.8466           21.25s
       747       55230.8466           21.16s
       748       55230.8466           21.07s
       749       55230.8466           20.99s
       750       55230.8466           20.90s
       751       55230.8466           20.81s
       752       55230.8466           20.72s
       753       55230.8466           20.63s
       754       55230.8466           20.55s
       755       55230.8466           20.46s
       756       55230.8466           20.36s
       757

       918       55230.8466            6.96s
       919       55230.8466            6.88s
       920       55230.8466            6.79s
       921       55230.8466            6.70s
       922       55230.8466            6.62s
       923       55230.8466            6.54s
       924       55230.8466            6.46s
       925       55230.8466            6.37s
       926       55230.8466            6.29s
       927       55230.8466            6.20s
       928       55230.8466            6.12s
       929       55230.8466            6.03s
       930       55230.8466            5.94s
       931       55230.8466            5.86s
       932       55230.8466            5.77s
       933       55230.8466            5.69s
       934       55230.8466            5.60s
       935       55230.8466            5.51s
       936       55230.8466            5.43s
       937       55230.8466            5.34s
       938       55230.8466            5.25s
       939       55230.8466            5.17s
       940

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=1000,
                           n_iter_no_change=None, presort='auto',
                           random_state=2, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=2,
                           warm_start=False)

In [181]:
y_pred=gbc.predict(X_test)

print(f1_score(y_test,y_pred))

confusion_matrix(y_test,y_pred)

0.2595419847328244


array([[34590,    29],
       [  165,    34]])